In [1]:
!pip install -U transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 56.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement huggingfacehub (from versions: none)
ERROR: No matching distribution found for huggingfacehub


In [2]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
from transformers import pipeline
from huggingface_hub import login
import os
from google.colab import userdata

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoProcessor,
    GenerationConfig,
    BitsAndBytesConfig,
)

In [4]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
login(os.environ["HF_TOKEN"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    load_in_4bit=True,
)
processor = AutoProcessor.from_pretrained(model_id)


config.json:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/89.4k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

In [6]:
url = "https://cs.stanford.edu/people/rak248/VG_100K_2/1.jpg"
image = Image.open(requests.get(url, stream=True).raw)

messages = [
    {"role": "user", "content": [
        {"type": "text", "text": "Respond to the following user query in a comprehensive and detailed way. You can write down your thought process before responding. Write your thoughts after 'Here is my thought process:' and write your response after 'Here is my response:'."},
        {"type": "image"},
        {"type": "text", "text": "What color is the clock? : "},

    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(image, input_text, return_tensors="pt").to(model.device)

output = model.generate(**inputs, max_new_tokens=200, num_return_sequences=2)
print(processor.decode(output[0]))
print(processor.decode(output[1]))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

Respond to the following user query in a comprehensive and detailed way. You can write down your thought process before responding. Write your thoughts after 'Here is my thought process:' and write your response after 'Here is my response:'.<|image|>What color is the clock? : <|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here is my thought process:

* The image shows a man standing on a sidewalk, holding a clock in his hand.
* The clock appears to be a traditional analog clock with a round face and hour and minute hands.
* The clock is not attached to a wall or any other object, suggesting that it is a portable timepiece.
* The clock's color is not explicitly visible, but based on the surrounding environment, it is likely to be a standard black or brown color.

Here is my response:

The clock is likely black or brown in color.<|eot_id|>
<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_h

In [17]:
url = "https://scienceqa-webdemo.s3.us-west-1.amazonaws.com/extra/8624/image.png"
image = Image.open(requests.get(url, stream=True).raw)

messages = [
    {"role": "user", "content": [
        {"type": "text", "text": "Which specific humidity level was measured within the outlined area shown? The map below shows humidity in the lower atmosphere on September 12, 2013. The map shows specific humidity, a measurement of the amount of water vapor in the air. The outlined area shows an air mass that influenced weather in Australia on that day. Look at the map. Then, answer the question below. Data source: United States National Oceanic and Atmospheric Administration/Earth System Research Laboratory, Physical Sciences Division : "},
        {"type": "image"},
        {"type": "text", "text":"Choices: 11 grams of water vapor per kilogram of air; 12 grams of water vapor per kilogram of air; 3 grams of water vapor per kilogram of air"},
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(image, input_text, return_tensors="pt").to(model.device)

output = model.generate(**inputs, max_new_tokens=200)
print(processor.decode(output[0]))

OutOfMemoryError: CUDA out of memory. Tried to allocate 80.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 35.06 MiB is free. Process 7819 has 14.71 GiB memory in use. Of the allocated memory 14.48 GiB is allocated by PyTorch, and 107.30 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)